<a href="https://colab.research.google.com/github/TheNeodev/enchaned-ultimate-rvc-/blob/main/notebooks/ultimate_rvc_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#   <h1 align="center"><b> Colab for [Ultimate RVC](https://github.com/JackismyShephard/ultimate-rvc) </b></h1>
<div align="center">

This Colab notebook will **help** you if you don’t have a GPU or if your PC isn’t very powerful.

Simply click `Runtime` in the top navigation bar and `Run all`. Wait for the output of the final cell to show the public gradio url and click on it.


---



**NOTE:** If Ultimate RVC is running too slowly with the default sharing method (gradio), consider instead selecting "ngrok" as sharing method.
This method requires supplying an access token, which you can find on [ngrok](https://ngrok.com/) after creating a personal account.
The ngrok link output in the final cell will lead you to a new site which will redirect you to Ultimate RVC.

In [ ]:
# @title 0: Initialize notebook
%pip install ipython-autotime pyngrok
%load_ext autotime

import codecs
import os
import time
import urllib.request
from pathlib import Path

import ipywidgets as widgets
from IPython.display import clear_output, display
from pyngrok import ngrok

clear_output()

In [ ]:
# @title 1: install dependencies


main_dir = "/content/HRVC"
light = codecs.decode("uggcf://nfgeny.fu/hi/0.6.3/vafgnyy.fu", "rot_13")
inits = codecs.decode("./fep/hygvzngr_eip/pber/znva.cl", "rot_13")
prerelease = "--prerelease if-necessary-or-explicit"


cloneing = codecs.decode(
    "uggcf://tvguho.pbz/WnpxvfzlFurcuneq/hygvzngr-eip.tvg",
    "rot_13",
)

!git clone $cloneing HRVC > /dev/null 2>&1
%cd {main_dir}

!apt install -y python3-dev unzip > /dev/null 2>&1
!curl -LsSf $light | sh > /dev/null 2>&1

os.environ["URVC_CONSOLE_LOG_LEVEL"] = "WARNING"
!uv run -q $prerelease $inits > /dev/null 2>&1
!uv add $prerelease matplotlib-inline==0.1.7 > /dev/null 2>&1


clear_output()

In [ ]:
# @title 2: Run Ultimate RVC
# @markdown  #### Choose a sharing method:

method = "gradio"  # @param ["gradio",  "ngrok", "cloudflared","localtunnel"]
ngrok_token = ""  # @param {type:"string"}
runpice = codecs.decode("./fep/hygvzngr_eip/jro/znva.cl", "rot_13")

if method == "gradio":
    !uv run $prerelease $runpice --share
elif method == "ngrok":
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(6969)
        print(f"Ngrok URL: {tunnel.public_url}")
        !uv run $prerelease $runpice --listen-port 6969
    except Exception as e:
        print(f"Error starting ngrok: {e}")
elif method == "cloudflared":
    !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb
    clear_output()
    !rm -rf nohup.out
    !nohup cloudflared tunnel --url localhost:6969 &
    clear_output()
    time.sleep(10)
    cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
    print(f"URVC Public URL: {cloudflare_url}")
    !uv run $prerelease $runpice --listen-port 6969
elif method == "localtunnel":
    !npm install -g localtunnel &>/dev/null
    Path("url.txt").open("w", encoding="utf-8").close()
    !lt --port 6969 >> url.txt 2>&1 &
    time.sleep(2)
    endpoint_ip = urllib.request.urlopen("https://ipv4.icanhazip.com").read().decode("utf8").strip("\n")
    tunnel_url = (
        Path("url.txt")
        .read_text(encoding="utf-8")
        .replace("your url is: ", "")
    )
    print(f"Share Link: {tunnel_url}")
    password_endpoint_widget = widgets.Text(
        value=endpoint_ip,
        description="Password IP:",
        disabled=True,
    )
    display(password_endpoint_widget)
    !uv run $prerelease $runpice --listen-port 6969


<div align="center">


# Extra Tabs


made by [NeoDev](https://github.com/TheNeodev)

In [ ]:



#@title mounth your Google Drive
#@markdown if your inputs audio on there
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title upload model to huggingface
#@markdown get your token on https://huggingface.co/settings/tokens
import huggingface_hub
import os
import tempfile
import zipfile

def upload_model(repo, pth, index, token, names_models):
    """
    Upload a model to the Hugging Face Hub by zipping the .pth and .index files and uploading the zip file along with a README.md.

    Args:
        repo: str, the name of the repository in the format "username/repo_name"
        pth: str, path to the model file (.pth)
        index: str, path to the index file (.index)
        token: str, the API token

    Returns:
        str, message indicating the success of the operation
    """
    # Validate input files
    if not os.path.isfile(pth):
        raise FileNotFoundError(f"Model file not found: {pth}")
    if not os.path.isfile(index):
        raise FileNotFoundError(f"Index file not found: {index}")

    # Extract model name from repo and define zip file name
    model_name = repo.split('/')[-1]
    zip_name = f"{model_name}.zip"

    # Initialize Hugging Face API and create repo if it doesn't exist
    api = huggingface_hub.HfApi()
    api.create_repo(token=token, repo_id=repo, exist_ok=True)

    # Create a temporary directory and zip the files
    with tempfile.TemporaryDirectory() as temp_dir:
        zip_path = os.path.join(temp_dir, f"{names_models}.zip")
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            # Add files to zip with their original names
            zipf.write(pth, arcname=os.path.basename(pth))
            zipf.write(index, arcname=os.path.basename(index))
        # Upload the zip file
        api.upload_file(
            token=token,
            path_or_fileobj=zip_path,
            path_in_repo=zip_name,
            repo_id=repo
        )
        print(f"Uploaded {zip_name}")

    # Define README content
    readme = f"""
# {model_name}
This is a model uploaded by [Ultimate RVC](https://github.com/JackismyShephard/ultimate-rvc), using [NeoDev](https://github.com/TheNeodev)'s script.
"""
    # Create and upload README
    with tempfile.NamedTemporaryFile(mode="w", delete=False) as tmpfile:
        tmpfile.write(readme)
        tmpfile_path = tmpfile.name
    api.upload_file(
        token=token,
        path_or_fileobj=tmpfile_path,
        path_in_repo="README.md",
        repo_id=repo
    )
    os.remove(tmpfile_path)
    print("Uploaded README.md")

    return "Model uploaded successfully"

# Example usage
repo = "user/names" #@param {type:"string"}
names_models = "my_model" #@param {type:"string"}

pth = "your model pth path" #@param {type:"string"}
index = "your model index" #@param {type:"string"}
token = "your token here" #@param {type:"string"}
upload_model(repo, pth, index, token, names_models)